In [1]:
import pandas as pd
import csv
import os
import json

### Document Loading  

(The json data is preprocessed offline)

In [2]:
from langchain.text_splitter import CharacterTextSplitter

datafile = './data/medical-notes.jsonl' 

with open(datafile, 'r') as file:
    json_list = [line.strip('{}\n') for line in file]


text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.create_documents(json_list[:100])

In [3]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=texts, 
                                    embedding=oembed, 
                                    collection_name="med_notes",
                                    persist_directory="med_db")

/tmp/ipykernel_74003/2578851399.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")


### Test the Vectorstore

In [4]:
question = "how to handle headache?"
results = vectorstore.similarity_search(question)
for res in results:
    print(f"{res.page_content} [{res.metadata}]")

"Question":"What can you do to prevent migraine with brainstem aura?","Answer":"That means you need to: Get enough sleep. Limit your stress. Exercise daily.","Url":"https:\/\/www.webmd.com\/migraines-headaches\/qa\/what-can-you-do-to-prevent-basilar-migraines","Title":"https:\/\/www.webmd.com\/migraines-headaches\/basilar-artery-migraines-causes-symptoms-tests-and-treatments" [{}]
"Question":"How can you change your diet to prevent migraine with brainstem aura?","Answer":"Diet can also affect migraines. Do these things: Eat a balanced diet. Avoid drugs and alcohol. Don't skip meals. Limit caffeine. Avoid any foods that have been triggers.","Url":"https:\/\/www.webmd.com\/migraines-headaches\/qa\/how-can-you-change-your-diet-to-prevent-basilar-migraines","Title":"https:\/\/www.webmd.com\/migraines-headaches\/basilar-artery-migraines-causes-symptoms-tests-and-treatments" [{}]
"Question":"Can migraine with brainstem aura be prevented?","Answer":"To keep from getting a MBA, it helps to avo